# Using API getting Census data
## Case of Ozo Coffee Shop
### Investigate the target audiance in which city in USA   

In [11]:
import pandas as pd
import requests
import json

In [12]:
#read the csv data
msas = 'msas.csv'
msas =  pd.read_csv(msas)

In [13]:
#transfer the column name into string
tableids = ['B01001', 'B19001', 'B03002']
tableidstring = ','.join(tableids)

In [14]:
# transfer geoid into string
geoid = msas['CBSA']
geoidstr = map(str, geoid)
geoidstring = ','.join(geoidstr)

In [15]:
#request url and loaded Json data
requesturl = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=%s' % (tableidstring) + '&geo_ids=310%7C01000US'
censusreporter=requests.get(requesturl)
loadedjson= json.loads(censusreporter.content)

In [24]:
# Look for income below 75K:
incomelist=[]
for klchiu in geoid:
    income_total= 0
    income_total += loadedjson['data']['31000US%s' % (klchiu)]['B19001']['estimate']['B19001011']#Income:$50,000 to $59,999
    income_total += loadedjson['data']['31000US%s' % (klchiu)]['B19001']['estimate']['B19001012']#Income:$$60,000 to $74,999
   
    incomelist.append(income_total)

#combine incomelist into msas dataframe. 
msas['Number of income below 75K'] = list(incomelist)

#Find the city which has a max population that income is below 75K
income_city = msas.loc[msas['Number of income below 75K'].idxmax()]

print(income_city)

CBSA                                                           35620
NAME                           New York-Newark-Jersey City, NY-NJ-PA
POPESTIMATE2017                                             20320876
Number of income below 75K                               1.07089e+06
Number of age from 18 to 34                              4.76974e+06
Number of Hispanic                                        4.7702e+06
income_percentage                                                  1
age_percentage                                                     1
hispanic_percentage                                          0.80412
mean_percentage_score                                       0.934707
Name: 249, dtype: object


In [21]:
#Look for the age between 18 and 34(male&female)
agelist=[]
for klchiu in geoid:
    age_total= 0
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001007']# male,18 and 19 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001008']# male,20 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001009']# male,21 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001010']# male,22 to 24 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001011']# male,25 to 29 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001012']# male,30 to 34 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001031']# female,18 and 19 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001032']# female,20 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001033']# female,21 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001034']# female,22 to 24 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001035']# female,25 to 29 years
    age_total += loadedjson['data']['31000US%s' % (klchiu)]['B01001']['estimate']['B01001036']# female,30 to 34 years
    
    agelist.append(age_total)

#combine agelist into msas dataframe. 
msas['Number of age from 18 to 34'] = list(agelist)

#Find the city which has a max population between age 18~34
age_city = msas.loc[msas['Number of age from 18 to 34'].idxmax()]

print(age_city)

CBSA                                                           35620
NAME                           New York-Newark-Jersey City, NY-NJ-PA
POPESTIMATE2017                                             20320876
Number of income below 75K                               1.07089e+06
Number of age from 18 to 34                              4.76974e+06
Number of Hispanic                                        4.7702e+06
income_percentage                                                  1
age_percentage                                                     1
hispanic_percentage                                          0.80412
mean_percentage_score                                       0.934707
Name: 249, dtype: object


In [27]:
#Hispanic
hispaniclist=[]
for klchiu in geoid:
    hispanic_total= 0
    hispanic_total += loadedjson['data']['31000US%s' % (klchiu)]['B03002']['estimate']['B03002012']
    
    hispaniclist.append(hispanic_total)
    
##combine hispaniclist into msas dataframe. 
msas['Number of Hispanic'] = list(hispaniclist)

##Find the city which has a max Hispanic population  
hispanic_city = msas.loc[msas['Number of Hispanic'].idxmax()]

print(hispanic_city)

CBSA                                                        31080
NAME                           Los Angeles-Long Beach-Anaheim, CA
POPESTIMATE2017                                          13353907
Number of income below 75K                                 702950
Number of age from 18 to 34                           3.36321e+06
Number of Hispanic                                     5.9322e+06
income_percentage                                        0.656414
age_percentage                                           0.705114
hispanic_percentage                                             1
mean_percentage_score                                    0.787176
Name: 210, dtype: object


In [23]:
##caculate the percentage of three categories
income_percentage = []
age_percentage = []
hispanic_percentage = []
for i, row in msas.iterrows():
    Percentage_Income = msas['Number of income below 75K'][i]/max(msas['Number of income below 75K'])
    Percentage_Age = msas['Number of age from 18 to 34'][i]/max(msas['Number of age from 18 to 34'])
    Percentage_Hispanic = msas['Number of Hispanic'][i]/max(msas['Number of Hispanic'])
    income_percentage.append(Percentage_Income)
    age_percentage.append(Percentage_Age)
    
    hispanic_percentage.append(Percentage_Hispanic)

#caculate the mean of three parameters
msas['income_percentage'] = income_percentage
msas['age_percentage'] = age_percentage
msas['hispanic_percentage'] = hispanic_percentage

mean_percentage_score = []
for i, row in msas.iterrows():
    Mean = (msas['income_percentage'][i] + msas['age_percentage'][i]+ msas['hispanic_percentage'][i])/3
    mean_percentage_score.append(Mean)
msas['mean_percentage_score'] = mean_percentage_score

#Find the city has the highest mean percentage across all three categories
higest_mean_city = msas.loc[msas['mean_percentage_score'].idxmax()]

print(higest_mean_city)

CBSA                                                           35620
NAME                           New York-Newark-Jersey City, NY-NJ-PA
POPESTIMATE2017                                             20320876
Number of income below 75K                               1.07089e+06
Number of age from 18 to 34                              4.76974e+06
Number of Hispanic                                        4.7702e+06
income_percentage                                                  1
age_percentage                                                     1
hispanic_percentage                                          0.80412
mean_percentage_score                                       0.934707
Name: 249, dtype: object
